In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
import bambi as bmb
import xarray as xr
import arviz as az
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=Warning)

In [2]:
%config InlineBackend.figure_format = 'retina'
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-viridish")

In [3]:
dat = pd.read_csv("surprise_correct_data.csv")
dat.head()

,subj_name,rt,zrt,CT,SC,BL,BF,subj_id,movie_id,lrt
0,1,728,0.037347,Congruent,Surprise,1,1,1,9_effect.mov,-3.287507
1,1,1178,1.770961,Congruent,Surprise,1,1,1,3_effect.mov,0.571522
2,1,1062,1.324074,Congruent,Surprise,1,1,1,8_effect.mov,0.280713
3,1,795,0.295463,Congruent,Surprise,1,1,1,7_effect.mov,-1.219213
4,1,444,-1.056756,Congruent,No Surprise,1,1,1,2_noeffect.mov,NaN


In [4]:
# surprise_cor_df = dat.loc[dat["experiment"] == "surprise"]
# surprise_cor_df.shape

In [13]:
mean = surprise_cor_df["rtTukey"].mean(skipna=True)
std = surprise_cor_df["rtTukey"].std(skipna=True)

surprise_cor_df["zrt"] = (surprise_cor_df["rtTukey"] - mean) / std

bf(zrt ~ is_congruent_trial * is_surprise_clip * blk +
       (1 + is_congruent_trial * blk | subj_id) +
       (1 | movie_id)
  ), 

In [31]:
d = surprise_cor_df[
    ["zrt", "is_congruent_trial", "is_surprise_clip", "block", "subj_id", "movie_id"]
]
df = d.dropna()
df.shape

(35609, 6)

In [6]:
m1 = bmb.Model(
    "zrt ~ CT * SC * BF + (1 + CT * SC * BF | subj_id) + (1 | movie_id)",
    dat,
    categorical="BF",
    family="asymmetriclaplace"
)

In [ ]:
fit1 = m1.fit(method="nuts_numpyro", idata_kwargs={"log_likelihood": True})

In [ ]:
az.plot_trace(fit1)